In [1]:
# Notebook for noodling with the bullet gym walker environments
# Implementation: https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/robot_locomotors.py

import pybullet
import pybullet_envs
import gym
import numpy as np
import matplotlib.pyplot as plt
import pdb
import time
human = 1

In [2]:
if human:
    env = gym.make('HumanoidBulletEnv-v0')
else:
    env = gym.make('Walker2DBulletEnv-v0')

env.render(mode="human")
obs = env.reset()
base_env = env.unwrapped

In [3]:
# Function definitions

# The goal here is to find a mirroring function for both our state and action space. 
# Going to start with the Walker2d environment, though getting a general solution for the pybullet locomotion envs would be idea
print('reset')
obs = env.reset()

def set_angles(joint_angles):
    for j,s in zip(base_env.ordered_joints, joint_angles):
        j.reset_position(s,0)

def mirror_more(more):
    z,sin_target,cos_target,vx,vy,vz,roll,pitch = more
    return [z, -sin_target, cos_target, vx, -vy, vz, -roll, -pitch]

def walker_mirror_obs(obs):
    m_more = mirror_more(obs[0:8])
    m_feet_contact = [obs[21],obs[20]]
    m_rgr = obs[8:14]
    m_grg = obs[14:20]
    return np.hstack([m_more,m_grg,m_rgr,m_feet_contact])

def walker_mirror_act(act):
    m_act = np.hstack([act[3:6],act[0:3]])
    return m_act

def human_mirror_obs(obs):
    m_more = mirror_more(obs[0:8])
    m_feet_contact = [obs[43],obs[42]]
    # all joints have [pos, velocity], so abs = 4 entries for 2 joints
    # abs   [ twist left, arch back ]  *relative to main torso, negate twist, arch stays same
    m_abd  = np.hstack( [-obs[8:10],obs[10:12] ]) 
    # center hip  [  roll left ]       *relative to abs, negate
    m_hip_c  = -obs[12:14] 
    # right hip   [ slide left,twist left,swing back ]    *relative to hip center
    # left hip    [ slide right, twist left, swing back ] *relative to hip center
    hip_r = obs[14:20]; m_hip_l  = hip_r
    hip_l = obs[20:26]; m_hip_r  = hip_l
    # right knee  [ bend forward ]                        *relative to right hip
    # left knee   [ bend forward ]                        *relative to left hip  
    # just swap them
    knee_r = obs[26:28]; m_knee_l = knee_r    
    knee_l = obs[28:30]; m_knee_r = knee_l       
    # right shldr [ pull in, rotate forward ]             *relative to main torso
    # left shldr  [ pull in, rotate backward ]            *relative to main torso
    # swap them, negate rotation.
    shld_r = obs[30:34]; m_shld_l = np.hstack([shld_r[0:2],-shld_r[2:4]])  
    shld_l = obs[34:38]; m_shld_r = np.hstack([shld_l[0:2],-shld_l[2:4]])
    # right elbow [ bend in ]                             *relative to right shoulder
    # left elbow  [ bend in ]                             *relative to left should
    # just swap them
    elb_r  = obs[38:40]; m_elb_l = elb_r
    elb_l  = obs[40:42]; m_elb_r = elb_l  
    
    m_obs = np.hstack([m_more,m_abd,m_hip_c,m_hip_r,m_knee_r,m_hip_l,m_knee_l,m_shld_r,m_elb_r,m_shld_l,m_elb_l,m_feet_contact])
    return m_obs

def human_mirror_act(act):
    # abs   [ twist left, arch back ]  *relative to main torso, negate twist, arch stays same
    m_abd = np.hstack([-act[0:1],act[1:2]])
    # center hip  [  roll left ]       *relative to abs, negate
    m_hip_c = -act[2:3]
    # right hip   [ slide left,twist left,swing back ]    *relative to hip center
    # left hip    [ slide right, twist left, swing back ] *relative to hip center
    # just swap them
    hip_r = act[3:6]; m_hip_l  = hip_r
    hip_l = act[6:9]; m_hip_r  = hip_l
    # right knee  [ bend forward ]                        *relative to right hip
    # left knee   [ bend forward ]                        *relative to left hip  
    # just swap them
    knee_r = obs[9:10];  m_knee_l = knee_r    
    knee_l = obs[10:11]; m_knee_r = knee_l       
    # right shldr [ pull in, rotate forward ]             *relative to main torso
    # left shldr  [ pull in, rotate backward ]            *relative to main torso
    # swap them, negate rotation.
    shld_r = obs[11:13]; m_shld_l = np.hstack([shld_r[0],-shld_r[1]]) 
    shld_l = obs[13:15]; m_shld_r = np.hstack([shld_l[0],-shld_l[1]])
    # right elbow [ bend in ]                             *relative to right shoulder
    # left elbow  [ bend in ]                             *relative to left should
    # just swap them
    elb_r  = obs[15:16]; m_elb_l = elb_r
    elb_l  = obs[16:17]; m_elb_r = elb_l
    m_act = np.hstack([m_abd,m_hip_c,m_hip_r,m_knee_r,m_hip_l,m_knee_l,m_shld_r,m_elb_r,m_shld_l,m_elb_l])
    return m_act



reset


In [5]:
# Test mirror functions
act = env.action_space.sample()
for _ in range(0,1):
    obs, reward, done, _ = env.step(act)
    env.render()
if human:
    m_obs = human_mirror_obs(obs)
    m_act = human_mirror_act(act)
else:     
    m_obs = walker_mirror_obs(obs)
    m_act = walker_mirror_act(act)
    
print("Before Mirror")
print(act.shape)
print(obs.shape)
print("After Mirror")
print(m_act.shape)
print(m_obs.shape)


Before Mirror
(17,)
(44,)
After Mirror
(17,)
(44,)


In [27]:
# mess with the joint angles here

env.reset()
if human:
# human
    # integer n = rotate by n*45 degrees
    abd    = [0,0]   # abs         [ twist left, arch back ]               *relative to main torso
    hip_c  = [0]     # center hip  [  roll left ]                          *relative to abs
    hip_r  = [0,0,0] # right hip   [ slide left,twist left,swing back ]    *relative to hip center
    hip_l  = [0,0,0] # left hip    [ slide right, twist left, swing back ] *relative to hip center
    knee_r = [0]     # right knee  [ bend forward ]                        *relative to right hip
    knee_l = [0]     # left knee   [ bend forward ]                        *relative to left hip  
    shld_r = [0,0]   # right shldr [ pull in, rotate forward ]             *relative to main torso
    shld_l = [0,0]   # left shldr  [ pull in, rotate backward ]            *relative to main torso
    elb_r  = [0]     # right elbow [ bend in ]                             *relative to right shoulder
    elb_l  = [0]     # left elbow  [ bend in ]                             *relative to left should
    
    joint_angles = np.pi/4 * np.hstack([abd,hip_c,hip_r,knee_r,hip_l,knee_l,shld_r,elb_r,shld_l,elb_l])
else:
    
# 2D walker (hip = abs, knee = relative to hip, ankle = relative to knee)
    gr = [0, 0,0]   #green-red [swing forward,bend forward,bend up]
    rg = [0, 0, 0]   #red-grean [swing forward,bend forward,bend up]
    joint_angles =  np.pi/4 * np.hstack([gr,rg])

# 0 state obs
# set_angles(0*joint_angles)
# obs2 = env.robot.calc_state()
# set the desired angles, save state to obs1
set_angles(joint_angles)
obs1 = env.robot.calc_state()
env.render()

print(obs1)
print(obs1.shape)
print("absolute (position, velocity)")
for j,s in zip(base_env.ordered_joints, joint_angles):
    print(j.current_position())

print(); print();
    
print("relative (position, velocity)")
for j,s in zip(base_env.ordered_joints, joint_angles):
    print(j.current_relative_position())
    
# we can go through and set the actions like the above, but probably better to use the high level gym interface (see next cell)
#joint_torques = [0,0,0,1,0,0]
#for j,a in zip(base_env.ordered_joints, joint_torques):
#    j.set_motor_torque(a)


[ 0.59999996  0.          1.          0.          0.          0.
  0.         -0.          0.          0.          0.42857143  0.
  0.          0.          0.6666666   0.          0.26315787  0.
  0.71428573  0.          1.0253165   0.          0.6666666   0.
  0.26315787  0.          0.71428573  0.          1.0253165   0.
  0.17241377  0.          0.17241377  0.          0.2857143   0.
 -0.17241377  0.         -0.17241377  0.          0.2857143   0.
  0.          0.        ]
(44,)
absolute (position, velocity)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)


relative (position, velocity)
(0.0, 0.0)
(0.4285714471569762, 0.0)
(0.0, 0.0)
(0.6666666524371061, 0.0)
(0.2631578644645371, 0.0)
(0.7142857073161337, 0.0)
(1.0253164540545934, 0.0)
(0.6666666524371061, 0.0)
(0.2631578644645371, 0.0)
(0.7142857073161337, 0.0)
(1.0253164540545934, 0.0)
(0.17241

In [31]:
# mess with the actions here
if human:
    # [ abd,  hip_c, hip_r,  knee_r,  hip_l,   knee_l,  shld_r,  elb_r,  shld_l,  elb_l]
    actions = np.array( [ 1,0,  0,  0,0,0,  0,  0,0,0,  0,  0,0,  0, 0,0,  1 ])
    m_actions = human_mirror_act(actions)
else:
    actions = np.array([0,0,0,0,0,0])
    m_actions = walker_mirror_act(actions)
    
# m_actions = mirror_act(actions)
state_hist = []
m_state_hist = []
action_hist = []
m_action_hist = []

env.reset()
for _ in range(50):
    obs, reward, done, _ = env.step(actions)
    state_hist.append(obs[0])
    env.render()
    time.sleep(0.05)

env.reset()
for _ in range(50):
    obs, reward, done, _ = env.step(m_actions)
    m_state_hist.append(obs[0])
    env.render()
    time.sleep(0.05)



In [ ]:
#plot previous cell
f = plt.figure(0,dpi = 60,figsize = (10,5))
plt.plot(range(0,len(state_hist)),state_hist)
plt.plot(range(0,len(m_state_hist)),m_state_hist)
plt.grid(True)
plt.tick_params(axis='both', which='major', labelsize=10)  
plt.show()

In [ ]:
# What's actually in each space?
## Mujoco's Walker2d has an observation space of Box(17,), But bullet has Box(22,)
# from line 75 https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/robot_locomotors.py
#
# state = np.clip(np.concatenate([more] + [j] + [self.feet_contact]), -5, +5)
#
#  more = np.array(
#        [
#            z - self.initial_z,
#            np.sin(angle_to_target),
#            np.cos(angle_to_target),
#            0.3 * vx,
#            0.3 * vy,
#            0.3 * vz,  # 0.3 is just scaling typical speed into -1..+1, no physical sense here
#            r,
#            p
#        ],
# 
# j is the relative joint positions
# j = np.array([j.current_relative_position() for j in self.ordered_joints],
#               dtype=np.float32).flatten()
#
# feet contact just tell us which foot is in contact with the ground
# env.robot.feet_contact